<center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">Vision</div></center>
<img src="./output.png" alt="Study.vision" style="width: 200px;"/>
<h1><center><div style="direction:rtl;font-family:B Lotus, B Nazanin, </div></center></h1>

In [1]:
import random
import numpy as np
import cv2
from keras.models import Sequential 
from keras.layers import Conv2D 
from keras.layers import MaxPooling2D 
from keras.layers import LSTM 
from keras.layers import Dense 
from keras.layers import Flatten 
from keras.layers import TimeDistributed

ImportError: cannot import name 'get_config' from 'tensorflow.python.eager.context' (C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\eager\context.py)

In [ ]:
size = 30

In [ ]:
# generate the next frame in the sequence 
def next_frame(last_step, column, size): 
    frame = np.zeros((size,size)) 
    # define the scope of the next step 
    lower = max(0, last_step-3) 
    upper = min(frame.shape[0]-1, last_step+3) 
    # choose the row index for the next step 
    step = random.randint(lower, upper) 
    # add the new step 
    cv2.circle(frame,(column, step),3,255,-1)
    #frame[step, column] = 1 
    return frame, step

In [ ]:
# generate a sequence of frames of a dot moving across an image 
def build_frames(size): 
    frames = list() 

    step = random.randint(0, size-1) 
    # decide if we are heading left or right 
    right = 1 if random.random() < 0.5 else 0 
    col = 0 if right else size-1 

    # create all frames 
    for i in range(0, size): 
        col = i if right else size-1-i 
        frame, step = next_frame(step, col, size) 
        frames.append(frame) 
    return frames, right

In [ ]:
# generate multiple sequences of frames and reshape for network input 
def generate_examples(size, n_patterns): 
    X, y = list(), list() 
    for _ in range(n_patterns): 
        frames, right = build_frames(size) 
        X.append(frames) 
        y.append(right) 
    # resize as [samples, timesteps, width, height, channels] 
    X = np.array(X).reshape(n_patterns, size, size, size, 1)
    y = np.array(y).reshape(n_patterns, 1) 
    return X, y

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

X, Y = generate_examples(size, 1) 
print("Right -->") if Y[0] else print("<-- Left")

for frame in X[0]:
    plt.imshow(frame[:,:,0], cmap="gray")
    plt.show()

In [ ]:
# define the model 
model = Sequential() 
model.add(TimeDistributed(Conv2D(2, (3,3), activation='relu'), 
                          input_shape=(None,size,size,1))) 
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2)))) 
model.add(TimeDistributed(Flatten())) 
model.add(LSTM(50)) 
model.add(Dense(1, activation='sigmoid')) 
print(model.summary())

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc']) 

In [ ]:
# fit model 
X, y = generate_examples(size, 1000) 
model.fit(X, y, batch_size=32, epochs=1)

In [ ]:
# evaluate model 
X, y = generate_examples(size, 100) 
loss, acc = model.evaluate(X, y, verbose=0) 
print('loss: %f, acc: %f' % (loss, acc*100))

In [ ]:
# prediction on new data 
X, y = generate_examples(size, 1) 
yhat = model.predict_classes(X, verbose=0) 
expected = "Right" if y[0]==1 else "Left" 
predicted = "Right" if yhat[0]==1 else "Left" 
print('Expected: %s, Predicted: %s' % (expected, predicted))